<a id='home'></a>
### Contents

[Imports](#Imports)

[Dataloading](#Dataloading)

<a id='Imports'></a>
### Imports
[back to contents](#home)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from os.path import join as p_join
import sys
from datetime import datetime, date, timedelta
from dotenv import load_dotenv
load_dotenv()
import typing as t
from typing import List, Tuple, Dict, Set, Any, Optional, Callable
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent))
import requests
from collections import Counter
import json
import pickle
from tqdm import tqdm
try:
    from fabulous import color as fb_color
    color_print = lambda x, color='green': print(getattr(fb_color, color)(x)) if 'fb_color' in globals() else print(x)
except Exception as e:
    color_print = lambda x, color='green': print(x)

from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
pd.set_option('display.max_columns', 200)

In [5]:
from src.parse_utils import fightInfoType, get_fight_name
from src.common_utils import get_container_ipaddr
from src.minio_utils import initialize_minio_client, load_json_from_minio
from src.db_utils import get_pg_engine
from src.processing import bfs_dict, minio_data_to_pandas, minio_data_to_postgres

<a id='Dataloading'></a>
### Dataloading
[back to contents](#home)

In [6]:
MINIO_ACCESS_KEY = os.environ['MINIO_ACCESS_KEY']
MINIO_SECRET_KEY = os.environ['MINIO_SECRET_KEY']
POSTGRES_USER = os.environ['POSTGRES_USER']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']

In [19]:
minio_client = initialize_minio_client(
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY
)

all_fights_list = load_json_from_minio(
    minio_client=minio_client,
    bucket_name='ufc-raw-data',
    object_name='ufc_stats.json',
    # version_id='9c478ec2-085e-42c2-a07f-771ec7e02df1'
)
print(f"len(all_fights_list): {len(all_fights_list):,}")

len(all_fights_list): 7,582


In [13]:
# date_string = dict(res.metadata)['Last-Modified']
# date_object = datetime.strptime(date_string, '%a, %d %b %Y %H:%M:%S %Z')

# print(date_object)


2024-03-20 00:04:23


In [20]:
all_fights_names = [get_fight_name(fight_info) for fight_info in all_fights_list]
[
    (k,v) for k, v in dict(Counter(all_fights_names)).items()
    if v > 1
]

[('UFC_Ultimate_Japan_Kazushi_Sakuraba_Marcus_Silveira_Yokohama_Kanagawa_Japan_December_21_1997',
  2)]

In [21]:
k = 15
res_dict = bfs_dict(one_fight_dict=all_fights_list[k])
res_dict

{'names__1': 'Nazim Sadykhov',
 'names__2': 'Evan Elder',
 'fight_name': 'lightweight bout',
 'method': "TKO - Doctor's Stoppage",
 'round': '3',
 'time': '0:38',
 'timeformat': '3 Rnd (5-5-5)',
 'referee': 'Mark Smith',
 'date': 'February 18, 2023',
 'location': 'Las Vegas, Nevada, USA',
 'event_name': 'UFC Fight Night: Andrade vs. Blanchfield',
 'event_uri': 'http://www.ufcstats.com/event-details/f21a3d68fb9df387',
 'fight_uri': 'http://www.ufcstats.com/fight-details/d45d9c7d60f332da',
 'winloose_fighter1': 'W',
 'winloose_fighter2': 'L',
 'per_round_stats__round1__fighter__1': 'Nazim Sadykhov',
 'per_round_stats__round1__fighter__2': 'Evan Elder',
 'per_round_stats__round1__kd__1': '0',
 'per_round_stats__round1__kd__2': '1',
 'per_round_stats__round1__sig_str__1': '28 of 74',
 'per_round_stats__round1__sig_str__2': '35 of 78',
 'per_round_stats__round1__sig_str_perc__1': '37%',
 'per_round_stats__round1__sig_str_perc__2': '44%',
 'per_round_stats__round1__totalstr__1': '28 of 74',


In [22]:
res_df = minio_data_to_pandas(all_fights_list=all_fights_list)
print(f"res_df: {res_df.shape[0]:,} x {res_df.shape[1]:,}")
res_df.head(3)

7582it [00:01, 3890.70it/s]


res_df: 7,582 x 165


,names__1,names__2,fight_name,method,round,time,timeformat,referee,date,location,event_name,event_uri,fight_uri,winloose_fighter1,winloose_fighter2,per_round_stats__round1__fighter__1,per_round_stats__round1__fighter__2,per_round_stats__round1__kd__1,per_round_stats__round1__kd__2,per_round_stats__round1__sig_str__1,per_round_stats__round1__sig_str__2,per_round_stats__round1__sig_str_perc__1,per_round_stats__round1__sig_str_perc__2,per_round_stats__round1__totalstr__1,per_round_stats__round1__totalstr__2,per_round_stats__round1__td_perc__1,per_round_stats__round1__td_perc__2,per_round_stats__round1__sub_att__1,per_round_stats__round1__sub_att__2,per_round_stats__round1__rev__1,per_round_stats__round1__rev__2,per_round_stats__round1__ctrl__1,per_round_stats__round1__ctrl__2,per_round_stats__round1__head__1,per_round_stats__round1__head__2,per_round_stats__round1__body__1,per_round_stats__round1__body__2,per_round_stats__round1__leg__1,per_round_stats__round1__leg__2,per_round_stats__round1__distance__1,per_round_stats__round1__distance__2,per_round_stats__round1__clinch__1,per_round_stats__round1__clinch__2,per_round_stats__round1__ground__1,per_round_stats__round1__ground__2,per_round_stats__round2__fighter__1,per_round_stats__round2__fighter__2,per_round_stats__round2__kd__1,per_round_stats__round2__kd__2,per_round_stats__round2__sig_str__1,per_round_stats__round2__sig_str__2,per_round_stats__round2__sig_str_perc__1,per_round_stats__round2__sig_str_perc__2,per_round_stats__round2__totalstr__1,per_round_stats__round2__totalstr__2,per_round_stats__round2__td_perc__1,per_round_stats__round2__td_perc__2,per_round_stats__round2__sub_att__1,per_round_stats__round2__sub_att__2,per_round_stats__round2__rev__1,per_round_stats__round2__rev__2,per_round_stats__round2__ctrl__1,per_round_stats__round2__ctrl__2,per_round_stats__round2__head__1,per_round_stats__round2__head__2,per_round_stats__round2__body__1,per_round_stats__round2__body__2,per_round_stats__round2__leg__1,per_round_stats__round2__leg__2,per_round_stats__round2__distance__1,per_round_stats__round2__distance__2,per_round_stats__round2__clinch__1,per_round_stats__round2__clinch__2,per_round_stats__round2__ground__1,per_round_stats__round2__ground__2,per_round_stats__round3__fighter__1,per_round_stats__round3__fighter__2,per_round_stats__round3__kd__1,per_round_stats__round3__kd__2,per_round_stats__round3__sig_str__1,per_round_stats__round3__sig_str__2,per_round_stats__round3__sig_str_perc__1,per_round_stats__round3__sig_str_perc__2,per_round_stats__round3__totalstr__1,per_round_stats__round3__totalstr__2,per_round_stats__round3__td_perc__1,per_round_stats__round3__td_perc__2,per_round_stats__round3__sub_att__1,per_round_stats__round3__sub_att__2,per_round_stats__round3__rev__1,per_round_stats__round3__rev__2,per_round_stats__round3__ctrl__1,per_round_stats__round3__ctrl__2,per_round_stats__round3__head__1,per_round_stats__round3__head__2,per_round_stats__round3__body__1,per_round_stats__round3__body__2,per_round_stats__round3__leg__1,per_round_stats__round3__leg__2,per_round_stats__round3__distance__1,per_round_stats__round3__distance__2,per_round_stats__round3__clinch__1,per_round_stats__round3__clinch__2,per_round_stats__round3__ground__1,per_round_stats__round3__ground__2,per_round_stats__round4__fighter__1,per_round_stats__round4__fighter__2,per_round_stats__round4__kd__1,per_round_stats__round4__kd__2,per_round_stats__round4__sig_str__1,per_round_stats__round4__sig_str__2,per_round_stats__round4__sig_str_perc__1,per_round_stats__round4__sig_str_perc__2,per_round_stats__round4__totalstr__1,per_round_stats__round4__totalstr__2,per_round_stats__round4__td_perc__1,per_round_stats__round4__td_perc__2,per_round_stats__round4__sub_att__1,per_round_stats__round4__sub_att__2,per_round_stats__round4__rev__1,per_round_stats__round4__rev__2,per_round_stats__round4__ctrl__1,per_round_stats__round4__ctrl__2,per_round_stats__round4__head__1,per_round_stats__round4__head__2,per_round_st

In [38]:
weight_groups = {
    "open weight": -20,
    "catch weight": -10,
    'flyweight': 1,
    'bantamweight': 2,
    'featherweight': 3,
    'lightweight': 4,
    'welterweight': 5,
    'middleweight': 6,
    'light heavyweight': 7,
    'heavyweight': 8,
    "women's strawweight": 100,
    "women's flyweight": 101,
    "women's bantamweight": 102
}

In [72]:
neighbouring_weight_groups_dict = {
    group: [gp for gp, wgt in weight_groups.items() if abs(wgt - main_wgt) <= 1] for 
    group, main_wgt in weight_groups.items()
}
neighbouring_weight_groups_dict

{'open weight': ['open weight'],
 'catch weight': ['catch weight'],
 'flyweight': ['flyweight', 'bantamweight'],
 'bantamweight': ['flyweight', 'bantamweight', 'featherweight'],
 'featherweight': ['bantamweight', 'featherweight', 'lightweight'],
 'lightweight': ['featherweight', 'lightweight', 'welterweight'],
 'welterweight': ['lightweight', 'welterweight', 'middleweight'],
 'middleweight': ['welterweight', 'middleweight', 'light heavyweight'],
 'light heavyweight': ['middleweight', 'light heavyweight', 'heavyweight'],
 'heavyweight': ['light heavyweight', 'heavyweight'],
 "women's strawweight": ["women's strawweight", "women's flyweight"],
 "women's flyweight": ["women's strawweight",
  "women's flyweight",
  "women's bantamweight"],
 "women's bantamweight": ["women's flyweight", "women's bantamweight"]}

In [73]:
res_df['fight_weight'] = res_df['fight_name'].apply(
    lambda x: 
    sorted([group_name for group_name in weight_groups if group_name in x])[-1] 
    if len([group_name for group_name in weight_groups if group_name in x]) >= 1 
    else None
)
names1_dct = (
    res_df
    .groupby(['fight_weight'])
    ['names__1']
    .agg(set)
    .to_dict()
)
names2_dct = (
    res_df
    .groupby(['fight_weight'])
    ['names__2']
    .agg(set)
    .to_dict()
)
fighters_by_weight_dict = {
    weight: set(names1_dct.get(weight, set())).union(names2_dct.get(weight, set())) 
    for weight in set(names1_dct.keys()).union(set(names2_dct.keys()))
}
del names1_dct, names2_dct
fighters_by_weight_dict

{'welterweight': {'AJ Fletcher',
  'Aaron Riley',
  'Aaron Simpson',
  'Abdul Razak Alhassan',
  'Abubakar Nurmagomedov',
  'Adam Fugitt',
  'Adlan Amagov',
  'Adrian Serrano',
  'Akihiro Gono',
  'Alan Jouban',
  'Albert Cheng',
  'Albert Tumenov',
  'Alberto Mina',
  'Aleksei Kunchenko',
  'Alex Garcia',
  'Alex Karalexis',
  'Alex Morono',
  'Alex Oliveira',
  'Alex Reyes',
  'Alexander Yakovlev',
  'Alexandre Barros',
  'Alvaro Herrera Mendoza',
  'Amilcar Alves',
  'Amir Sadollah',
  'Andre Fialho',
  'Andre Pederneiras',
  'Andreas Michailidis',
  'Andreas Stahl',
  'Andrew Craig',
  'Ange Loosa',
  'Ansar Chalangov',
  'Anthony Ivy',
  'Anthony Johnson',
  'Anthony Lapsley',
  'Anthony Pettis',
  'Anthony Rocco Martin',
  'Anthony Torres',
  'Anton Zafir',
  'Anying Wang',
  'Augusto Montano',
  'BJ Penn',
  'Bartosz Fabinski',
  'Bassil Hafez',
  'Belal Muhammad',
  'Ben Alloway',
  'Ben Askren',
  'Ben Earwood',
  'Ben Saunders',
  'Ben Wall',
  'Benji Radach',
  'Benoit Saint

In [76]:
fighters_clusters_lists = []
for neighbouring_weights in neighbouring_weight_groups_dict.values():
    current_cluster_set = set()
    for weight_group in neighbouring_weights:
        current_cluster_set = current_cluster_set.union(set(fighters_by_weight_dict[weight_group]))
    fighters_clusters_lists.append(list(current_cluster_set))
# Number of pairs it's reasonable to consider fights
print(f"{sum([len(cluster)*(len(cluster) - 1) / 2 for cluster in fighters_clusters_lists]):,}")

3,362,905.0


In [15]:
eng = get_pg_engine()
minio_data_to_postgres(
    all_fights_list=all_fights_list,
    schema='raw_data',
    table_name='all_fights_info',
    engine=eng
)
pd.read_sql('select * from raw_data.all_fights_info limit 3', eng)

7582it [00:01, 3804.08it/s]


,names__1,names__2,fight_name,method,round,time,timeformat,referee,date,location,event_name,event_uri,fight_uri,winloose_fighter1,winloose_fighter2,per_round_stats__round1__fighter__1,per_round_stats__round1__fighter__2,per_round_stats__round1__kd__1,per_round_stats__round1__kd__2,per_round_stats__round1__sig_str__1,per_round_stats__round1__sig_str__2,per_round_stats__round1__sig_str_perc__1,per_round_stats__round1__sig_str_perc__2,per_round_stats__round1__totalstr__1,per_round_stats__round1__totalstr__2,per_round_stats__round1__td_perc__1,per_round_stats__round1__td_perc__2,per_round_stats__round1__sub_att__1,per_round_stats__round1__sub_att__2,per_round_stats__round1__rev__1,per_round_stats__round1__rev__2,per_round_stats__round1__ctrl__1,per_round_stats__round1__ctrl__2,per_round_stats__round1__head__1,per_round_stats__round1__head__2,per_round_stats__round1__body__1,per_round_stats__round1__body__2,per_round_stats__round1__leg__1,per_round_stats__round1__leg__2,per_round_stats__round1__distance__1,per_round_stats__round1__distance__2,per_round_stats__round1__clinch__1,per_round_stats__round1__clinch__2,per_round_stats__round1__ground__1,per_round_stats__round1__ground__2,per_round_stats__round2__fighter__1,per_round_stats__round2__fighter__2,per_round_stats__round2__kd__1,per_round_stats__round2__kd__2,per_round_stats__round2__sig_str__1,per_round_stats__round2__sig_str__2,per_round_stats__round2__sig_str_perc__1,per_round_stats__round2__sig_str_perc__2,per_round_stats__round2__totalstr__1,per_round_stats__round2__totalstr__2,per_round_stats__round2__td_perc__1,per_round_stats__round2__td_perc__2,per_round_stats__round2__sub_att__1,per_round_stats__round2__sub_att__2,per_round_stats__round2__rev__1,per_round_stats__round2__rev__2,per_round_stats__round2__ctrl__1,per_round_stats__round2__ctrl__2,per_round_stats__round2__head__1,per_round_stats__round2__head__2,per_round_stats__round2__body__1,per_round_stats__round2__body__2,per_round_stats__round2__leg__1,per_round_stats__round2__leg__2,per_round_stats__round2__distance__1,per_round_stats__round2__distance__2,per_round_stats__round2__clinch__1,per_round_stats__round2__clinch__2,per_round_stats__round2__ground__1,per_round_stats__round2__ground__2,per_round_stats__round3__fighter__1,per_round_stats__round3__fighter__2,per_round_stats__round3__kd__1,per_round_stats__round3__kd__2,per_round_stats__round3__sig_str__1,per_round_stats__round3__sig_str__2,per_round_stats__round3__sig_str_perc__1,per_round_stats__round3__sig_str_perc__2,per_round_stats__round3__totalstr__1,per_round_stats__round3__totalstr__2,per_round_stats__round3__td_perc__1,per_round_stats__round3__td_perc__2,per_round_stats__round3__sub_att__1,per_round_stats__round3__sub_att__2,per_round_stats__round3__rev__1,per_round_stats__round3__rev__2,per_round_stats__round3__ctrl__1,per_round_stats__round3__ctrl__2,per_round_stats__round3__head__1,per_round_stats__round3__head__2,per_round_stats__round3__body__1,per_round_stats__round3__body__2,per_round_stats__round3__leg__1,per_round_stats__round3__leg__2,per_round_stats__round3__distance__1,per_round_stats__round3__distance__2,per_round_stats__round3__clinch__1,per_round_stats__round3__clinch__2,per_round_stats__round3__ground__1,per_round_stats__round3__ground__2,per_round_stats__round4__fighter__1,per_round_stats__round4__fighter__2,per_round_stats__round4__kd__1,per_round_stats__round4__kd__2,per_round_stats__round4__sig_str__1,per_round_stats__round4__sig_str__2,per_round_stats__round4__sig_str_perc__1,per_round_stats__round4__sig_str_perc__2,per_round_stats__round4__totalstr__1,per_round_stats__round4__totalstr__2,per_round_stats__round4__td_perc__1,per_round_stats__round4__td_perc__2,per_round_stats__round4__sub_att__1,per_round_stats__round4__sub_att__2,per_round_stats__round4__rev__1,per_round_stats__round4__rev__2,per_round_stats__round4__ctrl__1,per_round_stats__round4__ctrl__2,per_round_stats__round4__head__1,per_round_stats__round4__head__2,per_round_st

In [40]:
tmp_df = pd.read_sql(
    """
    with fighters as (
        select
            distinct names__1 as fighter
        from
            raw_data.all_fights_info
        union
            select
                distinct names__2 as fighter
            from
                raw_data.all_fights_info        
    )
    
    select
        f1.fighter as fighter1
        , f2.fighter as fighter2
    from
        fighters f1
        full outer join
            fighters f2
        on
            f1.fighter = f2.fighter
    where
        1 = 1
        and f1.fighter != f2.fighter
    """,
    eng
)
print(f"tmp_df: {tmp_df.shape[0]:,} x {tmp_df.shape[1]:,}")
tmp_df.head()

tmp_df: 0 x 2


,fighter1,fighter2
